# Import Libraries

In [1]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

^C


In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

^C


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests 
from bs4 import BeautifulSoup
import re

In [4]:
import pandas as pd
import numpy as np

# Instantiate Model

In [5]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Test Model

In [6]:
token = tokenizer.encode('Really nice food, shame about the hair in the dish though.', return_tensors='pt')

In [7]:
result = model(token)

In [8]:
result.logits

tensor([[-2.5593, -1.3775,  0.8409,  2.2248,  0.5476]],
       grad_fn=<AddmmBackward0>)

In [9]:
int(torch.argmax(result.logits))+1

4

# Scrape Reviews

In [10]:
yelp_link = r'https://www.yelp.co.uk/biz/the-pantry-amsterdam'

In [11]:
r = requests.get(yelp_link)
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [12]:
reviews[0]

'Our server was great!! Food was delicious. Very tiny restaurant in amsterdam super cozy. The 3 course special was good!'

In [13]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [14]:
df

,review
0,Our server was great!! Food was delicious. Ver...
1,"Dear Craig,What a gem! If you're looking for a..."
2,Great food and amazing service.I'm not a good ...
3,"Stoped in just to see if we could get a table,..."
4,The Pantry was the last meal we had in Amsterd...
5,"The Pantry is a cozy place. Yes , it's small b..."
6,Amazing experience! We didn't have a reservati...
7,What an unexpected treat! We wanted traditiona...
8,It is not easy to find a traditional dutch foo...
9,Ok. If you want the authentic Dutch experience...


In [15]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [16]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [17]:
sentiment_score(df['review'].iloc[0])

5

In [18]:
df['review'].iloc[2]

"Great food and amazing service.I'm not a good writer and don't know how to write hundreds of words in admiration of a place but I must say that it was better than expectations. I ordered a Tomato soup and Goat cheese croquettes for a warm starter and for the main course I had a steak. The quantity was good but I guess they can work a bit more on the quality of the ingredients. Also, the staff was very friendly and professional."

# Scrape Profile Names

In [19]:
r = requests.get(yelp_link)
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*user.*')
results = soup.find_all('a', {'href':regex})
profiles = [result.text for result in results]

In [20]:
profile_names = []
for name in profiles:
    if ". " in name:
        continue
    if "." in name:
        profile_names.append(name)

In [21]:
o = 1
for i in profile_names:
    print(str(o), ')', i )
    o = o + 1

1 ) Cindy C.
2 ) Jack C.
3 ) Emma W.
4 ) Daniel K.
5 ) Mia L.
6 ) Robin M.
7 ) Briana B.
8 ) Kathleen F.
9 ) Johanna S.
10 ) TS R.


In [22]:
profile_names

['Cindy C.',
 'Jack C.',
 'Emma W.',
 'Daniel K.',
 'Mia L.',
 'Robin M.',
 'Briana B.',
 'Kathleen F.',
 'Johanna S.',
 'TS R.']

In [23]:
df['name'] = profile_names

# Scrape Review Date

In [24]:
r = requests.get(yelp_link)
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*css-chan6m.*')
results = soup.find_all('span', {'class':regex})
review_date = [result.text for result in results]

In [25]:
df['date'] = review_date[:10]

# Add ID & Save to CSV

In [26]:
df['bar_id'] = 1

In [27]:
scraped_reviews = df.reindex(columns=['bar_id', 'date', 'name', 'review', 'sentiment'])
scraped_reviews

,bar_id,date,name,review,sentiment
0,1,10/4/2022,Cindy C.,Our server was great!! Food was delicious. Ver...,5
1,1,1/4/2022,Jack C.,"Dear Craig,What a gem! If you're looking for a...",5
2,1,17/3/2022,Emma W.,Great food and amazing service.I'm not a good ...,4
3,1,24/4/2022,Daniel K.,"Stoped in just to see if we could get a table,...",4
4,1,13/4/2022,Mia L.,The Pantry was the last meal we had in Amsterd...,5
5,1,14/4/2022,Robin M.,"The Pantry is a cozy place. Yes , it's small b...",4
6,1,11/4/2022,Briana B.,Amazing experience! We didn't have a reservati...,5
7,1,15/4/2022,Kathleen F.,What an unexpected treat! We wanted traditiona...,5
8,1,6/8/2021,Johanna S.,It is not easy to find a traditional dutch foo...,4
9,1,4/5/2022,TS R.,Ok. If you want the authentic Dutch experience...,3


In [28]:
scraped_reviews.to_csv(r'C:\Users\jesse\Sentiment Analysis Project\reviews.csv', index=0)